Reference: 
1. SpinalNet: Deep Neural Network with Gradual Input
2. https://github.com/dipuk0506/SpinalNet

# 1.0 Read test image 

In [ ]:
import torchvision
from torchvision import transforms
import torch.utils.data as data
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

batch_size_train = 64

TRANSFORM_IMG = transforms.Compose([transforms.Resize((240, 320)), transforms.ToTensor()])

# load images in batches
TRAIN_DATA_PATH = "regression/train/"
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)



# 2.0 Train Model

In [ ]:
# model: regression
# first_HL = 50
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

torch.backends.cudnn.enabled = False

first_HL = 50

class Net_R(nn.Module):

  def __init__(self):
      super(Net_R, self).__init__()
      # kernal 5, maxpooling = 2
      self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
      self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
      self.conv2_drop = nn.Dropout2d()
      # features shape = 20*57*77
      self.lru = nn.LeakyReLU()
      self.fc1 = nn.Linear(int((20*57*77)/2), first_HL) 
      self.fc2 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc3 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc4 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc5 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc6 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fcx = nn.Linear(first_HL*6, 1)
        

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 20*57*77)
    x1 = x[:, 0:int((20*57*77)/2)]
    
    x1 = self.lru(self.fc1(x1))
    x2 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x1], dim=1)
    x2 = self.lru(self.fc2(x2))
    x3 = torch.cat([ x[:,0:int((20*57*77)/2)], x2], dim=1)
    x3 = self.lru(self.fc3(x3))
    x4 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x3], dim=1)
    x4 = self.lru(self.fc4(x4))
    x5 = torch.cat([ x[:,0:int((20*57*77)/2)], x4], dim=1)
    x5 = self.lru(self.fc3(x5))
    x6 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x5], dim=1)
    x6 = self.lru(self.fc4(x6))

    # before fc2
    x = torch.cat([x1, x2], dim=1)
    x = torch.cat([x, x3], dim=1)
    x = torch.cat([x, x4], dim=1)
    x = torch.cat([x, x5], dim=1)
    x = torch.cat([x, x6], dim=1)

    # fc2
    x = self.fcx(x)

    return x

In [1]:
# model2: learning_rate = 0.01
# training

from math import sqrt
import pandas as pd
import shutil

model2 = Net_R()
model2.cuda()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()
EPOCH = 100
result = pd.DataFrame(columns = ['epoch', 'train_loss'])

for i in range(1, EPOCH+1):
  for batch_idx, (data, target) in enumerate(train_data_loader):
    data, target = data.cuda(), target.cuda()
    output = model2(data)
    loss = loss_func(output, target.type(torch.cuda.FloatTensor)) 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(batch_idx)
    print(loss.item())

  joblib_file = 'model2_epoch_' + str(i) + '.pt'
  torch.save(model2, joblib_file)
    
  shutil.move(os.getcwd() + '\\' + joblib_file, os.getcwd() + '\\newRegression')
  
  result.loc[len(result)] = [i, sqrt(loss.item())]
  filename = 'model2_result_' + str(i) + '.csv'
  result.to_csv(filename, encoding='utf-8', index=False)